### Working with missing values

It occurs rather frequently that DataFrames are incomplete regarding the content of all data rows. There will probably be no occasion in which complete rows are entered empty, however it may very well occur that some part of an observation is missing.<br>
The question is how to deal with this situation? Let's have a look at the following DataFrame.

In [18]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
data_dir = os.path.join(Path(os.getcwd()).parents[2], "data", "ntbk_data", "04_data")
data = {'col1': [1,2,3,None,5], 'col2' : ['a',np.nan,'c','d','b'], 'col3' : [10,20,30,np.nan,np.nan], 
        'col4' : [np.nan,np.nan,np.nan,np.nan,0.6], 'col5' : [np.nan,np.nan,np.nan,np.nan,np.nan], 
        'col6' : ['a','b','c','d','e'], 'col7' : [10,25,40,np.nan,15]}

df = pd.DataFrame.from_dict(data)

In [19]:
df

col1 col2  col3  col4  col5 col6  col7
0   1.0    a  10.0   NaN   NaN    a  10.0
1   2.0  NaN  20.0   NaN   NaN    b  25.0
2   3.0    c  30.0   NaN   NaN    c  40.0
3   NaN    d   NaN   NaN   NaN    d   NaN
4   5.0    b   NaN   0.6   NaN    e  15.0

In [20]:
df.isnull()

col1   col2   col3   col4  col5   col6   col7
0  False  False  False   True  True  False  False
1  False   True  False   True  True  False  False
2  False  False  False   True  True  False  False
3   True  False   True   True  True  False   True
4  False  False   True  False  True  False  False

In [21]:
df.col1.isnull()#return a Series

0    False
1    False
2    False
3     True
4    False
Name: col1, dtype: bool

As you can see, we entered some 'missing' values on purpose, in this case, the Python __None__ type as well as the __np.nan__ type are converted and handled as __np.nan__ in the pandas DataFrame. The first thing you should do is check the DataFrame for these values so you have an overview of the completeness of your data.
Of course, you can use the __.describe()__ function here and compare the values but checking per column is more thorough.

In [22]:
#combining isnull() with sum() results in a nice overview
df.isnull().sum()

col1    1
col2    1
col3    2
col4    4
col5    5
col6    0
col7    1
dtype: int64

### Removing  data from a DataFrame


What to do if data is missing in a DataFrame? One way is to remove observations (rows) or attributes (columns) completely. This can be done if you are sure that it does not harm your information harvesting overall!<br>
Have a look at our DataFrame - it is easy to see that column 5 does not carry any valuable informartion whatsoever - let's remove it!

In [23]:
df.drop('col5', axis = 1, inplace=True)
#in such a case you may want to know, where the data came from after all or if there was an import error!

In [24]:
df

col1 col2  col3  col4 col6  col7
0   1.0    a  10.0   NaN    a  10.0
1   2.0  NaN  20.0   NaN    b  25.0
2   3.0    c  30.0   NaN    c  40.0
3   NaN    d   NaN   NaN    d   NaN
4   5.0    b   NaN   0.6    e  15.0

One may argue, whether there is much information contained in __col4__ as well, but let's leave it for the time being.

In [25]:
#this command removes any column where a np.nan value is found; 
#omitting the axis parameter results in the same behaviour for rows
df1 = df.dropna(axis = 1)
df1

col6
0    a
1    b
2    c
3    d
4    e

### Imputing data

We can try to fill in the missing data using the __fillna()__ function. This makes sense for numerical data. Let's try this for a single column.

In [26]:
df.col1.fillna(4.0, inplace=True)
df

col1 col2  col3  col4 col6  col7
0   1.0    a  10.0   NaN    a  10.0
1   2.0  NaN  20.0   NaN    b  25.0
2   3.0    c  30.0   NaN    c  40.0
3   4.0    d   NaN   NaN    d   NaN
4   5.0    b   NaN   0.6    e  15.0

We don't always want to guess the correct value, neither do we want to inspect each entry specifically, nor is this feasible if the DataFrame is large! A good measure may be to use the mean value in a column. Whether or not this makes sense depends on your knowledge of the data!

In [27]:
print(df.col7.mean())

22.5


In [28]:
df.col7.fillna(df.col7.mean(), inplace=True)
df

col1 col2  col3  col4 col6  col7
0   1.0    a  10.0   NaN    a  10.0
1   2.0  NaN  20.0   NaN    b  25.0
2   3.0    c  30.0   NaN    c  40.0
3   4.0    d   NaN   NaN    d  22.5
4   5.0    b   NaN   0.6    e  15.0

In [29]:
df.describe()

col1  col3  col4       col7
count  5.000000   3.0   1.0   5.000000
mean   3.000000  20.0   0.6  22.500000
std    1.581139  10.0   NaN  11.456439
min    1.000000  10.0   0.6  10.000000
25%    2.000000  15.0   0.6  15.000000
50%    3.000000  20.0   0.6  22.500000
75%    4.000000  25.0   0.6  25.000000
max    5.000000  30.0   0.6  40.000000

Using __describe()__ on the column gives a description on the numerical data in the DataFrame:
- count: the number of non-null elements in this column
- mean: the mean value of the column
- std: the standard deviation
- min/max: min and max value respectively
- 25%/50%/75%: The respective percentile

In [30]:
df.col2.describe()#applied on non-numerical data

count     4
unique    4
top       a
freq      1
Name: col2, dtype: object

Applied on a column, we can retrieve:
- count: the number of values
- the number of unique values
- the top value
- the frequency of the top value

In [31]:
df.col7.describe()

count     5.000000
mean     22.500000
std      11.456439
min      10.000000
25%      15.000000
50%      22.500000
75%      25.000000
max      40.000000
Name: col7, dtype: float64

### Correlation in data

Here is a link to some [housing data](https://www.kaggle.com/schirmerchad/bostonhoustingmlnd) in the Boston suburbian area. The column __RM__ describes the size in square metres per real estate, the __MEDV__ column refers to the value of the object in question.

In [32]:
dfh = pd.read_csv(os.path.join(data_dir,'housing.csv'), encoding= 'utf-8')
dfh.head()

RM  LSTAT  PTRATIO      MEDV
0  6.575   4.98     15.3  504000.0
1  6.421   9.14     17.8  453600.0
2  7.185   4.03     17.8  728700.0
3  6.998   2.94     18.7  701400.0
4  7.147   5.33     18.7  760200.0

Using the __corr()__ function we can quickly establish correlation factors for all variables and it is obvious to see that there is a positive correlation between the size of an object and its value, as we expected!

In [33]:
dfh.corr()

RM     LSTAT   PTRATIO      MEDV
RM       1.000000 -0.612033 -0.304559  0.697209
LSTAT   -0.612033  1.000000  0.360445 -0.760670
PTRATIO -0.304559  0.360445  1.000000 -0.519034
MEDV     0.697209 -0.760670 -0.519034  1.000000

### Working with data


Let's work a bit with a concrete data set and apply our knowledge about data engineering / data wrangling to answer some conrete questions! Here is a [link](https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016) to a data set describing suicides rates in various countries. We import the data into a DataFrame and take an initial glimpse at it.

In [34]:
dfS = pd.read_csv(os.path.join(data_dir, 'suicide_data.csv'))

In [35]:
#rename some column names
dfS.rename(columns = {' gdp_for_year ($) ':'gdp_year', 'gdp_per_capita ($)':'gdp_cap'}, inplace = True)

In [36]:
#if you are unsure about the meaning of the specified data, look it up under the link!
dfS.head()

country  year     sex          age  suicides_no  population  \
0  Albania  1987    male  15-24 years           21      312900   
1  Albania  1987    male  35-54 years           16      308000   
2  Albania  1987  female  15-24 years           14      289700   
3  Albania  1987    male    75+ years            1       21800   
4  Albania  1987    male  25-34 years            9      274300   

   suicides/100k pop country-year  HDI for year       gdp_year  gdp_cap  \
0               6.71  Albania1987           NaN  2,156,624,900      796   
1               5.19  Albania1987           NaN  2,156,624,900      796   
2               4.83  Albania1987           NaN  2,156,624,900      796   
3               4.59  Albania1987           NaN  2,156,624,900      796   
4               3.28  Albania1987           NaN  2,156,624,900      796   

        generation  
0     Generation X  
1           Silent  
2     Generation X  
3  G.I. Generation  
4          Boomers

In [37]:
dfS.describe()

year   suicides_no    population  suicides/100k pop  \
count  27820.000000  27820.000000  2.782000e+04       27820.000000   
mean    2001.258375    242.574407  1.844794e+06          12.816097   
std        8.469055    902.047917  3.911779e+06          18.961511   
min     1985.000000      0.000000  2.780000e+02           0.000000   
25%     1995.000000      3.000000  9.749850e+04           0.920000   
50%     2002.000000     25.000000  4.301500e+05           5.990000   
75%     2008.000000    131.000000  1.486143e+06          16.620000   
max     2016.000000  22338.000000  4.380521e+07         224.970000   

       HDI for year        gdp_cap  
count   8364.000000   27820.000000  
mean       0.776601   16866.464414  
std        0.093367   18887.576472  
min        0.483000     251.000000  
25%        0.713000    3447.000000  
50%        0.779000    9372.000000  
75%        0.855000   24874.000000  
max        0.944000  126352.000000

__Grouping__ allows for aggregation over certain columns, e.g. iw want to know all suicide values per country, we can sum all the entries with respect to each country. 

In [39]:
dfGroup = dfS.groupby(['country']).sum(numeric_only=True)
dfGroup

year  suicides_no  population  suicides/100k pop  \
country                                                                    
Albania               527796         1970    62325467             924.76   
Antigua and Barbuda   647832           11     1990228             179.14   
Argentina             744000        82219  1035985431            3894.59   
Armenia               596832         1905    77348173             976.21   
Aruba                 336720          101     1259677            1596.52   
...                      ...          ...         ...                ...   
United Arab Emirates  144540          622    36502275              94.89   
United Kingdom        744000       136805  1738767780            2790.92   
United States         744000      1034013  8054027201            5140.97   
Uruguay               672072        13138    84068943            6538.96   
Uzbekistan            528348        34803   486422532            2138.17   

                      HDI for year   gdp_cap  
country                                       
Albania                     32.304    490788  
Antigua and Barbuda         28.140   3385212  
Argentina                   93.552   2944044  
Armenia                     66.252    558428  
Aruba                        0.000   4069236  
...                            ...       ...  
United Arab Emirates        19.800   3035664  
United Kingdom             103.620  11869908  
United States              106.992  14608296  
Uruguay                     80.628   2561016  
Uzbekistan                  54.600    257712  

[101 rows x 6 columns]

Other forms of aggregation are:
- count: number of non-null observations
- sum:	Sum of values
- mean:	Mean of values
- mad:	Mean absolute deviation
- median: Arithmetic median of values
- min:	Minimum
- max:	Maximum
- mode:	Mode
- abs:	Absolute Value
- prod:	Product of values
- std:	standard deviation
- var:  variance
- sem: standard error of the mean
- quantile:	Sample quantile (value at %)
- cumsum:	Cumulative sum
- cumprod:	Cumulative product
- cummax:	Cumulative maximum
- cummin:	Cumulative minimum

If we want to retrieve parts from our DataFrame we can achieve this by using comparion: Certain values of columns which match our defined criteria.   

In [40]:
#retrieve a DataFrame listing all rows which match the string 'Germany' in the column country
#or: give me all the data regarding Germany
dfG = dfS[dfS.country == 'Germany']
dfG.head()

country  year     sex          age  suicides_no  population  \
9710  Germany  1990    male    75+ years         1516     1717700   
9711  Germany  1990    male  55-74 years         2406     6593100   
9712  Germany  1990    male  35-54 years         3302    11127100   
9713  Germany  1990  female    75+ years         1174     3978800   
9714  Germany  1990    male  25-34 years         1488     6721200   

      suicides/100k pop country-year  HDI for year           gdp_year  \
9710              88.26  Germany1990         0.801  1,764,967,948,917   
9711              36.49  Germany1990         0.801  1,764,967,948,917   
9712              29.68  Germany1990         0.801  1,764,967,948,917   
9713              29.51  Germany1990         0.801  1,764,967,948,917   
9714              22.14  Germany1990         0.801  1,764,967,948,917   

      gdp_cap       generation  
9710    23546  G.I. Generation  
9711    23546  G.I. Generation  
9712    23546           Silent  
9713    23546  G.I. Generation  
9714    23546          Boomers

In [41]:
dfG.describe()

year  suicides_no    population  suicides/100k pop  \
count   312.000000   312.000000  3.120000e+02         312.000000   
mean   2002.500000   933.532051  6.489986e+06          15.559904   
std       7.512048   886.297355  3.178586e+06          17.260059   
min    1990.000000     5.000000  1.482300e+06           0.120000   
25%    1996.000000   168.250000  4.373980e+06           3.532500   
50%    2002.500000   759.000000  4.922470e+06          10.595000   
75%    2009.000000  1249.500000  9.352304e+06          20.867500   
max    2015.000000  3427.000000  1.314881e+07          88.260000   

       HDI for year       gdp_cap  
count    108.000000    312.000000  
mean       0.881778  35164.230769  
std        0.040699   8716.933587  
min        0.801000  23546.000000  
25%        0.855000  27888.000000  
50%        0.906000  32783.500000  
75%        0.915000  43614.000000  
max        0.916000  50167.000000

In [43]:
#TASK: retrieve all data of male persons in Germany from the year 2000
dfS.loc[(dfS.country == 'Germany') & (dfS.sex == 'male') & (dfS.year == 2000)]


country  year   sex          age  suicides_no  population  \
9830  Germany  2000  male    75+ years         1069     1738251   
9831  Germany  2000  male  55-74 years         2528     8760853   
9832  Germany  2000  male  35-54 years         2937    12201076   
9834  Germany  2000  male  25-34 years         1002     6118772   
9835  Germany  2000  male  15-24 years          575     4709057   
9840  Germany  2000  male   5-14 years           25     4562649   

      suicides/100k pop country-year  HDI for year           gdp_year  \
9830              61.50  Germany2000         0.855  1,949,953,934,034   
9831              28.86  Germany2000         0.855  1,949,953,934,034   
9832              24.07  Germany2000         0.855  1,949,953,934,034   
9834              16.38  Germany2000         0.855  1,949,953,934,034   
9835              12.21  Germany2000         0.855  1,949,953,934,034   
9840               0.55  Germany2000         0.855  1,949,953,934,034   

      gdp_cap       generation  
9830    24922  G.I. Generation  
9831    24922           Silent  
9832    24922          Boomers  
9834    24922     Generation X  
9835    24922     Generation X  
9840    24922       Millenials

In [44]:
#TASK: retrieve all data of male persons in Germany from the year 2000
t1 = dfG[(dfG.sex == 'male') & (dfG.year == 2000)]
t1

country  year   sex          age  suicides_no  population  \
9830  Germany  2000  male    75+ years         1069     1738251   
9831  Germany  2000  male  55-74 years         2528     8760853   
9832  Germany  2000  male  35-54 years         2937    12201076   
9834  Germany  2000  male  25-34 years         1002     6118772   
9835  Germany  2000  male  15-24 years          575     4709057   
9840  Germany  2000  male   5-14 years           25     4562649   

      suicides/100k pop country-year  HDI for year           gdp_year  \
9830              61.50  Germany2000         0.855  1,949,953,934,034   
9831              28.86  Germany2000         0.855  1,949,953,934,034   
9832              24.07  Germany2000         0.855  1,949,953,934,034   
9834              16.38  Germany2000         0.855  1,949,953,934,034   
9835              12.21  Germany2000         0.855  1,949,953,934,034   
9840               0.55  Germany2000         0.855  1,949,953,934,034   

      gdp_cap       generation  
9830    24922  G.I. Generation  
9831    24922           Silent  
9832    24922          Boomers  
9834    24922     Generation X  
9835    24922     Generation X  
9840    24922       Millenials

In [45]:
t1.describe()

year  suicides_no    population  suicides/100k pop  HDI for year  \
count     6.0     6.000000  6.000000e+00           6.000000  6.000000e+00   
mean   2000.0  1356.000000  6.348443e+06          23.928333  8.550000e-01   
std       0.0  1136.868682  3.667596e+06          20.865353  1.216188e-16   
min    2000.0    25.000000  1.738251e+06           0.550000  8.550000e-01   
25%    2000.0   681.750000  4.599251e+06          13.252500  8.550000e-01   
50%    2000.0  1035.500000  5.413914e+06          20.225000  8.550000e-01   
75%    2000.0  2163.250000  8.100333e+06          27.662500  8.550000e-01   
max    2000.0  2937.000000  1.220108e+07          61.500000  8.550000e-01   

       gdp_cap  
count      6.0  
mean   24922.0  
std        0.0  
min    24922.0  
25%    24922.0  
50%    24922.0  
75%    24922.0  
max    24922.0

Task: Retrieve all data onwards from 2000 regarding [Generation X](https://en.wikipedia.org/wiki/Generation_X). What are the countries involved?

In [53]:
gen_x_older2000 = dfS.loc[(dfS.year > 2000) & (dfS.generation == 'Generation X')]
gen_x_older2000.country.unique()

array(['Albania', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Barbados', 'Belarus', 'Belgium', 'Belize',
       'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Cabo Verde',
       'Canada', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba',
       'Cyprus', 'Czech Republic', 'Denmark', 'Ecuador', 'El Salvador',
       'Estonia', 'Fiji', 'Finland', 'France', 'Georgia', 'Germany',
       'Greece', 'Grenada', 'Guatemala', 'Guyana', 'Hungary', 'Iceland',
       'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Kazakhstan',
       'Kiribati', 'Kuwait', 'Kyrgyzstan', 'Latvia', 'Lithuania',
       'Luxembourg', 'Maldives', 'Malta', 'Mauritius', 'Mexico',
       'Mongolia', 'Montenegro', 'Netherlands', 'New Zealand',
       'Nicaragua', 'Norway', 'Oman', 'Panama', 'Paraguay', 'Philippines',
       'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Republic of Korea',
       'Romania', 'Russian Federation', 

In [54]:
t2 = dfS[(dfS.year > 2000) & (dfS.generation == 'Generation X')]
t2.head()

country  year     sex          age  suicides_no  population  \
144  Albania  2001    male  25-34 years           22      206484   
154  Albania  2001  female  25-34 years            4      222771   
157  Albania  2002    male  25-34 years           23      206286   
163  Albania  2002  female  25-34 years            7      223685   
174  Albania  2003    male  25-34 years            9      205433   

     suicides/100k pop country-year  HDI for year       gdp_year  gdp_cap  \
144              10.65  Albania2001           NaN  4,060,758,804     1451   
154               1.80  Albania2001           NaN  4,060,758,804     1451   
157              11.15  Albania2002           NaN  4,435,078,648     1573   
163               3.13  Albania2002           NaN  4,435,078,648     1573   
174               4.38  Albania2003           NaN  5,746,945,913     2021   

       generation  
144  Generation X  
154  Generation X  
157  Generation X  
163  Generation X  
174  Generation X

In [55]:
t2.country.unique()

array(['Albania', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Barbados', 'Belarus', 'Belgium', 'Belize',
       'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Cabo Verde',
       'Canada', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba',
       'Cyprus', 'Czech Republic', 'Denmark', 'Ecuador', 'El Salvador',
       'Estonia', 'Fiji', 'Finland', 'France', 'Georgia', 'Germany',
       'Greece', 'Grenada', 'Guatemala', 'Guyana', 'Hungary', 'Iceland',
       'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Kazakhstan',
       'Kiribati', 'Kuwait', 'Kyrgyzstan', 'Latvia', 'Lithuania',
       'Luxembourg', 'Maldives', 'Malta', 'Mauritius', 'Mexico',
       'Mongolia', 'Montenegro', 'Netherlands', 'New Zealand',
       'Nicaragua', 'Norway', 'Oman', 'Panama', 'Paraguay', 'Philippines',
       'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Republic of Korea',
       'Romania', 'Russian Federation', 

In [56]:
#Task: retrieve all data from the DataFrame regarding Generation X in Germany between 2000 and 2010 (excluding both)

dfS.loc[(dfS.generation == 'Generation X') & (dfS.country == 'Germany') & (dfS.year > 2000) & (dfS.year < 2010)]






country  year     sex          age  suicides_no  population  \
9846  Germany  2001    male  25-34 years          985     5850255   
9850  Germany  2001  female  25-34 years          221     5552605   
9858  Germany  2002    male  25-34 years          903     5614445   
9862  Germany  2002  female  25-34 years          219     5348715   
9870  Germany  2003    male  25-34 years          868     5393875   
9874  Germany  2003  female  25-34 years          225     5161703   
9882  Germany  2004    male  25-34 years          820     5196875   
9886  Germany  2004  female  25-34 years          180     5000272   
9894  Germany  2005    male  25-34 years          730     5056621   
9898  Germany  2005  female  25-34 years          171     4889216   
9906  Germany  2006    male  25-34 years          629     4963413   
9910  Germany  2006  female  25-34 years          154     4819150   
9917  Germany  2007    male  25-34 years          654     4913538   
9922  Germany  2007  female  25-34 years          145     4782312   
9930  Germany  2008    male  25-34 years          632     4906502   
9934  Germany  2008  female  25-34 years          157     4780395   
9941  Germany  2009    male  25-34 years          637     4909814   
9946  Germany  2009  female  25-34 years          146     4784538   

      suicides/100k pop country-year  HDI for year           gdp_year  \
9846              16.84  Germany2001           NaN  1,950,648,769,575   
9850               3.98  Germany2001           NaN  1,950,648,769,575   
9858              16.08  Germany2002           NaN  2,079,136,081,310   
9862               4.09  Germany2002           NaN  2,079,136,081,310   
9870              16.09  Germany2003           NaN  2,505,733,634,312   
9874               4.36  Germany2003           NaN  2,505,733,634,312   
9882              15.78  Germany2004           NaN  2,819,245,095,605   
9886               3.60  Germany2004           NaN  2,819,245,095,605   
9894              14.44  Germany2005         0.887  2,861,410,272,354   
9898               3.50  Germany2005         0.887  2,861,410,272,354   
9906              12.67  Germany2006           NaN  3,002,446,368,084   
9910               3.20  Germany2006           NaN  3,002,446,368,084   
9917              13.31  Germany2007           NaN  3,439,953,462,907   
9922               3.03  Germany2007           NaN  3,439,953,462,907   
9930              12.88  Germany2008           NaN  3,752,365,607,148   
9934               3.28  Germany2008           NaN  3,752,365,607,148   
9941              12.97  Germany2009           NaN  3,418,005,001,389   
9946               3.05  Germany2009           NaN  3,418,005,001,389   

      gdp_cap    generation  
9846    24874  Generation X  
9850    24874  Generation X  
9858    26441  Generation X  
9862    26441  Generation X  
9870    31816  Generation X  
9874    31816  Generation X  
9882    35772  Generation X  
9886    35772  Generation X  
9894    36289  Generation X  
9898    36289  Generation X  
9906    38088  Generation X  
9910    38088  Generation X  
9917    43667  Generation X  
9922    43667  Generation X  
9930    47702  Generation X  
9934    47702  Generation X  
9941    43570  Generation X  
9946    43570  Generation X

In [57]:
#Task: retrieve all data from the DataFrame regarding Generation X in Germany between 2000 and 2010 (excluding both)
t3 = t2[(t2.country == 'Germany') & (t2.year > 2000) & (t2.year < 2010)]
t3

country  year     sex          age  suicides_no  population  \
9846  Germany  2001    male  25-34 years          985     5850255   
9850  Germany  2001  female  25-34 years          221     5552605   
9858  Germany  2002    male  25-34 years          903     5614445   
9862  Germany  2002  female  25-34 years          219     5348715   
9870  Germany  2003    male  25-34 years          868     5393875   
9874  Germany  2003  female  25-34 years          225     5161703   
9882  Germany  2004    male  25-34 years          820     5196875   
9886  Germany  2004  female  25-34 years          180     5000272   
9894  Germany  2005    male  25-34 years          730     5056621   
9898  Germany  2005  female  25-34 years          171     4889216   
9906  Germany  2006    male  25-34 years          629     4963413   
9910  Germany  2006  female  25-34 years          154     4819150   
9917  Germany  2007    male  25-34 years          654     4913538   
9922  Germany  2007  female  25-34 years          145     4782312   
9930  Germany  2008    male  25-34 years          632     4906502   
9934  Germany  2008  female  25-34 years          157     4780395   
9941  Germany  2009    male  25-34 years          637     4909814   
9946  Germany  2009  female  25-34 years          146     4784538   

      suicides/100k pop country-year  HDI for year           gdp_year  \
9846              16.84  Germany2001           NaN  1,950,648,769,575   
9850               3.98  Germany2001           NaN  1,950,648,769,575   
9858              16.08  Germany2002           NaN  2,079,136,081,310   
9862               4.09  Germany2002           NaN  2,079,136,081,310   
9870              16.09  Germany2003           NaN  2,505,733,634,312   
9874               4.36  Germany2003           NaN  2,505,733,634,312   
9882              15.78  Germany2004           NaN  2,819,245,095,605   
9886               3.60  Germany2004           NaN  2,819,245,095,605   
9894              14.44  Germany2005         0.887  2,861,410,272,354   
9898               3.50  Germany2005         0.887  2,861,410,272,354   
9906              12.67  Germany2006           NaN  3,002,446,368,084   
9910               3.20  Germany2006           NaN  3,002,446,368,084   
9917              13.31  Germany2007           NaN  3,439,953,462,907   
9922               3.03  Germany2007           NaN  3,439,953,462,907   
9930              12.88  Germany2008           NaN  3,752,365,607,148   
9934               3.28  Germany2008           NaN  3,752,365,607,148   
9941              12.97  Germany2009           NaN  3,418,005,001,389   
9946               3.05  Germany2009           NaN  3,418,005,001,389   

      gdp_cap    generation  
9846    24874  Generation X  
9850    24874  Generation X  
9858    26441  Generation X  
9862    26441  Generation X  
9870    31816  Generation X  
9874    31816  Generation X  
9882    35772  Generation X  
9886    35772  Generation X  
9894    36289  Generation X  
9898    36289  Generation X  
9906    38088  Generation X  
9910    38088  Generation X  
9917    43667  Generation X  
9922    43667  Generation X  
9930    47702  Generation X  
9934    47702  Generation X  
9941    43570  Generation X  
9946    43570  Generation X

In [66]:
#TASK (based on t2): List the suicide rates of the richest (per capita) 10% of all the countries between [2000,2010]
#make use of the .quantile() function

t2_richest_suic_rates = t2.loc[(t2.year > 2000) & (t2.year < 2010) & (t2.gdp_cap > t2.gdp_cap.quantile(0.9)), ['country', 'suicides/100k pop']]

print(t2_richest_suic_rates.head(10))
t2_richest_suic_rates.country.unique()


      country  suicides/100k pop
2066  Austria              15.85
2071  Austria               3.34
7565  Denmark              14.61
7570  Denmark               4.24
7577  Denmark              16.13
7582  Denmark               4.34
7589  Denmark              14.06
7594  Denmark               2.65
7602  Denmark              10.72
7606  Denmark               3.89


array(['Austria', 'Denmark', 'Finland', 'Iceland', 'Ireland',
       'Luxembourg', 'Netherlands', 'Norway', 'Qatar', 'San Marino',
       'Singapore', 'Sweden', 'Switzerland', 'United Kingdom'],
      dtype=object)

In [49]:
#TASK (based on t2): List the suicide rates of the richest (per capita) 10% of all the countries between [2000,2010]
#make use of the .quantile() function
top10 = t2[(t2.year > 2000) & (t2.year < 2010) & (t2.gdp_cap > t2.gdp_cap.quantile(0.9))]
top10.country.unique()

array(['Austria', 'Denmark', 'Finland', 'Iceland', 'Ireland',
       'Luxembourg', 'Netherlands', 'Norway', 'Qatar', 'San Marino',
       'Singapore', 'Sweden', 'Switzerland', 'United Kingdom'],
      dtype=object)

In [50]:
top10.head()

country  year     sex          age  suicides_no  population  \
2066  Austria  2008    male  25-34 years           86      542606   
2071  Austria  2008  female  25-34 years           18      539635   
7565  Denmark  2006    male  25-34 years           52      355908   
7570  Denmark  2006  female  25-34 years           15      353444   
7577  Denmark  2007    male  25-34 years           56      347246   

      suicides/100k pop country-year  HDI for year         gdp_year  gdp_cap  \
2066              15.85  Austria2008           NaN  430,294,287,388    54294   
2071               3.34  Austria2008           NaN  430,294,287,388    54294   
7565              14.61  Denmark2006           NaN  282,884,912,894    55354   
7570               4.24  Denmark2006           NaN  282,884,912,894    55354   
7577              16.13  Denmark2007           NaN  319,423,370,134    62229   

        generation  
2066  Generation X  
2071  Generation X  
7565  Generation X  
7570  Generation X  
7577  Generation X

In [67]:
#TASK: solve the same question for the poorest 10%


t2_poor_suic_rates = t2.loc[(t2.year > 2000) & (t2.year < 2010) & (t2.gdp_cap < t2.gdp_cap.quantile(0.1)), ['country', 'suicides/100k pop']]

print(t2_poor_suic_rates.head(10))
t2_poor_suic_rates.country.unique()



      country  suicides/100k pop
144   Albania              10.65
154   Albania               1.80
157   Albania              11.15
163   Albania               3.13
174   Albania               4.38
175   Albania               4.04
182   Albania               7.85
186   Albania               4.95
1099  Armenia               0.70
1101  Armenia               0.36


array(['Albania', 'Armenia', 'Azerbaijan', 'Belarus', 'Bulgaria',
       'Colombia', 'Ecuador', 'El Salvador', 'Fiji', 'Georgia',
       'Guatemala', 'Guyana', 'Kazakhstan', 'Kiribati', 'Kyrgyzstan',
       'Montenegro', 'Paraguay', 'Philippines', 'Romania',
       'Russian Federation', 'Serbia', 'Sri Lanka', 'Suriname',
       'Thailand', 'Turkmenistan', 'Ukraine', 'Uzbekistan'], dtype=object)

In [71]:
#TASK: In Germany - for GenX, are the suicides rates higher for men or women?

t2.loc[t2.country == 'Germany'].groupby(['sex']).mean(numeric_only=True).loc[:, 'suicides/100k pop']







sex
female     4.735625
male      16.611875
Name: suicides/100k pop, dtype: float64

In [51]:
gM = t3[t3.sex=='male']
gM.suicides_no.sum()

6858

In [52]:
gF = t3[t3.sex=='female']
gF.suicides_no.sum()

1618

In [78]:
#TASK: Which generation has the highest abolute suicide rates in Germany?

dfG.groupby(['generation', 'year'], as_index = False)['suicides_no'].sum().groupby(['generation']).max()
#dfS.loc[dfS.country == 'Germany'].groupby(['generation']).max(numeric_only=True).loc[:,'suicides/100k pop']








year  suicides_no
generation                        
Boomers          2015         6561
G.I. Generation  2000         6513
Generation X     2015         4322
Generation Z     2015           28
Millenials       2015         1455
Silent           2015         5303

In [73]:
dfG.head()

country  year     sex          age  suicides_no  population  \
9710  Germany  1990    male    75+ years         1516     1717700   
9711  Germany  1990    male  55-74 years         2406     6593100   
9712  Germany  1990    male  35-54 years         3302    11127100   
9713  Germany  1990  female    75+ years         1174     3978800   
9714  Germany  1990    male  25-34 years         1488     6721200   

      suicides/100k pop country-year  HDI for year           gdp_year  \
9710              88.26  Germany1990         0.801  1,764,967,948,917   
9711              36.49  Germany1990         0.801  1,764,967,948,917   
9712              29.68  Germany1990         0.801  1,764,967,948,917   
9713              29.51  Germany1990         0.801  1,764,967,948,917   
9714              22.14  Germany1990         0.801  1,764,967,948,917   

      gdp_cap       generation  
9710    23546  G.I. Generation  
9711    23546  G.I. Generation  
9712    23546           Silent  
9713    23546  G.I. Generation  
9714    23546          Boomers

In [74]:
df1 = dfG.groupby(['generation','year'], as_index=False)['suicides_no'].sum().groupby(['generation']).max()
#(['A', 'B'], as_index=False)['C'].sum()
df1

year  suicides_no
generation                        
Boomers          2015         6561
G.I. Generation  2000         6513
Generation X     2015         4322
Generation Z     2015           28
Millenials       2015         1455
Silent           2015         5303